In [15]:
import requests
import zipfile
from pathlib import Path

SOURCE_URL = "https://www.data.gouv.fr/api/1/datasets/r/3cf88405-3362-417e-9dda-d93c31880b93"
SOURCE_PAGE = "https://www.data.gouv.fr/datasets/repertoire-operationnel-des-metiers-et-des-emplois-rome/"

BASE_DIR = Path("./data/raw/temps/rome")
ZIP_PATH = BASE_DIR / "RefRomeXml.zip"
EXTRACT_DIR = BASE_DIR / "extracted"

BASE_DIR.mkdir(parents=True, exist_ok=True)
EXTRACT_DIR.mkdir(parents=True, exist_ok=True)

print("📥 Téléchargement du dataset ROME : 'Répertoire Opérationnel des Métiers et des Emplois (ROME)'")
print(f"🔗 Source officielle : {SOURCE_PAGE}")

r = requests.get(SOURCE_URL)
r.raise_for_status()

with open(ZIP_PATH, "wb") as f:
    f.write(r.content)

print(f"✅ ZIP téléchargé : {ZIP_PATH}")

# ============================================================
# UNZIP
# ============================================================

xml_file_path = None

print("📦 Extraction du ZIP...")

with zipfile.ZipFile(ZIP_PATH, "r") as zip_ref:
    for name in zip_ref.namelist():
        # if name.endswith("unix_fiche_emploi_metier_v460_iso8859-15.xml"):
        # if name.endswith("unix_fiche_emploi_metier_v460.json"):
        if name.endswith("unix_fiche_emploi_metier_v460_iso8859-15.xml"):
            zip_ref.extract(name, EXTRACT_DIR)
            xml_file_path = EXTRACT_DIR / name
            break

if not xml_file_path:
    raise FileNotFoundError("❌ Fichier XML ROME attendu non trouvé dans l’archive")

print(f"📄 Fichier XML détecté : {xml_file_path}")


# ============================================================
# XML → JSON
# ============================================================

import xmltodict
import json

print("🔄 Conversion XML → JSON...")

with open(xml_file_path, "r", encoding="iso-8859-15") as f:
    data = xmltodict.parse(f.read())

json_path = "rome.json"

with open(json_path, "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

print(f"✅ JSON généré : {json_path}")

📥 Téléchargement du dataset ROME : 'Répertoire Opérationnel des Métiers et des Emplois (ROME)'
🔗 Source officielle : https://www.data.gouv.fr/datasets/repertoire-operationnel-des-metiers-et-des-emplois-rome/
✅ ZIP téléchargé : data\raw\temps\rome\RefRomeXml.zip
📦 Extraction du ZIP...
📄 Fichier XML détecté : data\raw\temps\rome\extracted\unix_fiche_emploi_metier_v460_iso8859-15.xml
🔄 Conversion XML → JSON...
✅ JSON généré : rome.json


In [16]:
def ensure_list(x):
    if x is None:
        return []
    if isinstance(x, list):
        return x
    return [x]

import json

with open("rome.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [17]:
import xmltodict
import json
import polars as pl

print("Nombre de fiches métiers %s" % len(data['fiches_metier']['fiche_metier']))

records = []

for i, e in enumerate(data['fiches_metier']['fiche_metier']):
    if i >= 0:
        # print("i = ", i)
        # print("e = ", e)
        # print(e['appellations']['appellation'])
        if e['appellations'] is not None and e['appellations']['appellation'] is not None:
            for a in e['appellations']['appellation']:
                if isinstance(a, str) or a is None:
                    continue
                records.append({
                    "code_naf": None,
                    "code_rome": e['rome']['code_rome'],
                    "name": e['rome']['intitule'],
                    "desc": a['libelle_court'] + ' (code OGR:' + a['code_ogr'] +') libelle:' + a['libelle']
                })

df = pl.DataFrame(records)
print(df.sample(20))
df.write_csv("rome.csv")

Nombre de fiches métiers 1584
shape: (20, 4)
┌──────────┬───────────┬─────────────────────────────────┬─────────────────────────────────┐
│ code_naf ┆ code_rome ┆ name                            ┆ desc                            │
│ ---      ┆ ---       ┆ ---                             ┆ ---                             │
│ null     ┆ str       ┆ str                             ┆ str                             │
╞══════════╪═══════════╪═════════════════════════════════╪═════════════════════════════════╡
│ null     ┆ I1605     ┆ Mécanicien / Mécanicienne en m… ┆ Mécanicien / Mécanicienne diés… │
│ null     ┆ C1110     ┆ Souscripteur / Souscriptrice e… ┆ Souscripteur / Souscriptrice I… │
│ null     ┆ G1209     ┆ Animateur / Animatrice de lois… ┆ Professeur / Professeure de yo… │
│ null     ┆ K1726     ┆ Pisteur / Pisteuse secouriste   ┆ Tireur d'avalancheur / Tireuse… │
│ null     ┆ H3202     ┆ Monteur régleur / Monteuse rég… ┆ Opérateur(trice) monteur(se) i… │
│ …        ┆ …         ┆ 